In [1]:
from collections import defaultdict
import re

In [12]:
def read_input(infile):
    required = defaultdict(list)
    steps = set()
    with open(infile, 'r') as inf:
        for line in inf.readlines():
            m = re.match(r'Step (\w) must be finished before step (\w) can begin.', line.strip()).groups()
            steps.add(m[0])
            steps.add(m[1])
            required[m[1]].append(m[0])
    return sorted(list(steps)), required

In [44]:
def order(steps, required):
    available = []
    for s in steps:
        if s not in required:
            available.append(s)

    while len(available) != len(steps):
        next_available = []
        for s in steps:
            if s in available:
                continue
            all_available = True
            for rs in required[s]:
                if rs not in available:
                    all_available = False
                    break
            if all_available:
                next_available.append(s)
                break

        available += sorted(next_available)

        if len(next_available) == 0:
            raise Exception('At the end??')
        
    return ''.join(available)

def time_req(steps, required, t_base, n_workers):
    available = []
    finished_steps = defaultdict(list)
    startable = []
    running = []
    avail_workers = n_workers
    
    for s in steps:
        if s not in required:
            startable.append(s)

    t = 0
    while len(available) != len(steps):
    # for _ in range(20):

        # print('t', t, 'w', avail_workers, 'f', finished_steps[t], 'r', running, 'a', available)
        if len(finished_steps[t]) != 0:
            avail_workers += len(finished_steps[t])
            for s in finished_steps[t]:
                # print('Finished', s)
                running.remove(s)
            available += finished_steps[t]

            for s in steps:
                if s in available or s in startable or s in running:
                    continue
            
                all_available = True
                for rs in required[s]:
                    if rs not in available:
                        all_available = False
                        break
                if all_available:
                    startable.append(s)

        started = []
        for s in startable:
            if avail_workers > 0:
                t_work = ord(s) - ord('A') + 1 + t_base
                # print(s, t_work)
                avail_workers -= 1
                finished_steps[t + t_work].append(s)
                started.append(s)

        # print('started', started)
        for s in started:
            startable.remove(s)
            running.append(s)

        t += 1
        
    return t-1


In [43]:
print('*******\nPuzzle1\n*******\n')

print('Test case\n---------\n')

res = order(*read_input('input07a.txt'))

print(f'Order is {res}')

assert res == 'CABDFE'

print('\nPuzzle case\n-----------\n')

res = order(*read_input('input07.txt'))

print(f'Order is {res}')

assert res == 'GKPTSLUXBIJMNCADFOVHEWYQRZ'

print('\n*******\nPuzzle2\n*******\n')

print('Test case\n---------\n')

res = time_req(*read_input('input07a.txt'), 0, 2)

print(f'Time required is {res}')

assert res == 15

print('\nPuzzle case\n-----------\n')

res = time_req(*read_input('input07.txt'), 60, 5)

print(f'Time required is {res}')

assert res == 920


*******
Puzzle1
*******

Test case
---------

Order is CABDFE

Puzzle case
-----------

Order is GKPTSLUXBIJMNCADFOVHEWYQRZ

*******
Puzzle2
*******

Test case
---------

Time required is 15

Puzzle case
-----------

Time required is 920
